In [74]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 75 (delta 33), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (75/75), done.


In [8]:
ls

drive/  Mecab-ko-for-Google-Colab/  sample_data/


In [9]:
cd Mecab-ko-for-Google-Colab/

/content/Mecab-ko-for-Google-Colab


In [10]:
!bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 92kB 8.3MB/s 
     |████████████████████████████████| 460kB 48.1MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-12-16 09:55:57--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff00::22c2:513, 2406:da00:ff00::22e9:9f55, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.141.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/download

In [75]:
from matplotlib import rcParams, pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, GlobalMaxPooling1D, Conv1D, Dropout, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from konlpy.tag import Mecab
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
from konlpy.tag import Okt
import tqdm
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.snowball import SnowballStemmer
from sklearn.linear_model import LogisticRegression
import matplotlib.font_manager as fm
from future.utils import iteritems
from collections import Counter
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import gensim
# import fasttext
import warnings
import nltk
warnings.filterwarnings(action='ignore')

In [76]:
train = pd.read_csv("/content/drive/My Drive/AI 진짜 뉴스/dataset/news_train.csv", index_col=0,  encoding="utf-8")
test = pd.read_csv("/content/drive/My Drive/AI 진짜 뉴스/dataset/news_test.csv", index_col=0)
sample_submission = pd.read_csv("/content/drive/My Drive/AI 진짜 뉴스/dataset/sample_submission.csv")

In [21]:
train['content'].head

<bound method NDFrame.head of n_id
NEWS02580    [이데일리 MARKETPOINT]15:32 현재 코스닥 기관 678억 순매도
NEWS02580         "실적기반" 저가에 매집해야 할 8월 급등유망주 TOP 5 전격공개
NEWS02580             하이스탁론, 선취수수료 없는 월 0.4% 최저금리 상품 출시
NEWS02580              종합 경제정보 미디어 이데일리 - 무단전재 & 재배포 금지
NEWS09727                          전국적인 소비 붐 조성에 기여할 예정
                                ...                    
NEWS01333           미 FDA 임상3상 허가 임박. 묻고 따블로 갈 바이오 황제주.
NEWS01333                       똑똑해진 소비자..한국도 이젠 소형차 시대
NEWS01333                       똑똑해진 소비자..한국도 이젠 소형차 시대
NEWS01333                   2020년 한국 TV 2대중 1대 인터넷 연결된다
NEWS01333                   2020년 한국 TV 2대중 1대 인터넷 연결된다
Name: content, Length: 118745, dtype: object>

In [27]:
train_clear = []
for text in train['content']:
  temp = re.sub("[%&-\[\].,!?\"'·#:;~()]", '', text)
  # print("clear : ",temp)
  train_clear.append(temp)
train['content'] = train_clear

test_clear = []
for text in test['content']:
  temp = re.sub("[%&-\[\].,!?\"'·#:;~()]", '', text)
  # print("clear : ",temp)
  test_clear.append(temp)
test['content'] = test_clear
# del test_clear, train_clear

In [28]:
tokenizer = Mecab()

In [30]:
# ttx = []
def text_preprocessing(text_list, tokenizer):
    
    stopwords = ['을', '를', '이', '가', '은', '는', 'null']
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','을','으로','자','에','와','한','하다', 'null']

    token_list = []
    for text in tqdm.tqdm(text_list):
        txt = re.sub('[^가-힣a-z]', ' ', text.lower())
        # ttx.append(txt)
        token = tokenizer.morphs(txt)
        token = [t for t in token if t not in stopwords ]
        token_list.append(token)

    return token_list

In [31]:
trn_tokenlist = text_preprocessing(train['content'], tokenizer) 
tst_tokenlist = text_preprocessing(test['content'], tokenizer) 

100%|██████████| 142565/142565 [00:19<00:00, 7209.70it/s]


In [32]:
print(trn_tokenlist[:5])

[['데일리', '현재', '코스닥', '기관', '억', '순매도'], ['실적', '기반', '저가', '매집', '해야', '할', '월', '급등', '유망주', '전격', '공개'], ['하이스', '탁론', '선취', '수수료', '없', '월', '최저', '금리', '상품', '출시'], ['종합', '경제', '정보', '미디어', '데', '일리', '무단', '전재', '재', '배포', '금지'], ['전국', '적', '인', '소비', '붐', '조성', '기여', '할', '예정']]


In [48]:
train_join = []
test_join = []
# trn_tokenlist:
for tkn in tqdm.tqdm(trn_tokenlist):
  str = ' '.join(tkn)
  train_join.append(str)
train['content'] = train_join

for tkn in tqdm.tqdm(tst_tokenlist):
  str = ' '.join(tkn)
  test_join.append(str)
test['content'] = test_join


100%|██████████| 142565/142565 [00:00<00:00, 851364.48it/s]


In [49]:
# print(train_join[:5])
train.head()

,date,title,content,ord,info
n_id,,,,,
NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,데일리 현재 코스닥 기관 억 순매도,1,0
NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,실적 기반 저가 매집 해야 할 월 급등 유망주 전격 공개,2,1
NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,하이스 탁론 선취 수수료 없 월 최저 금리 상품 출시,3,1
NEWS02580,20200605,[마감]코스닥 기관 678억 순매도,종합 경제 정보 미디어 데 일리 무단 전재 재 배포 금지,4,0
NEWS09727,20200626,"롯데·공영 등 7개 TV 홈쇼핑들, 동행세일 동참",전국 적 인 소비 붐 조성 기여 할 예정,1,0


In [50]:
test.head()

,date,title,content,ord,id
n_id,,,,,
NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",마이크 로 장비 양산 돌입 전방 업체 투자 확대 로 본업 호조 연일 신고,1,NEWS00237_1
NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",데일리 김대웅 기자 반도체 장비 업체 코 세스 기술력 마이크 로 발광 다이오드 시대...,2,NEWS00237_2
NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",최근 대형 업체 거래 맺 고 관련 장비 양산 돌입 하 면서 주가 연일 신고 달리 고...,3,NEWS00237_3
NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",필두 로 올해 부터 마이크로 시대 본격 적 개 화 할 것 예상 되 면서 수주 더욱 ...,4,NEWS00237_4
NEWS00237,20200118,"[주목!e스몰캡]코세스, 마이크로LED 시장 개화 최대수혜 기대",코 세스 반도체 장비 제조 판매 하 업체 로 지난 년 월 코스닥 시장 상장 했 고 ...,5,NEWS00237_5


In [51]:
cnt = np.array(train['info'])
print("가짜 뉴스 수 : ",np.sum(cnt))
print("진짜 뉴스 수 : ",train['info'].shape - np.sum(cnt))

가짜 뉴스 수 :  46932
진짜 뉴스 수 :  [71813]


In [52]:
# 

# info가 1이면 가짜뉴스
true = []
fake = []
# for info in tqdm.tqdm(train['info']):
for content, info in tqdm.tqdm(zip(train['content'], train['info'])):
  if info == 0:
    true.append(content)
  else:
    fake.append(content)

118745it [00:00, 1327580.99it/s]


In [53]:
print(true[:5])
print(fake[:5])

['데일리 현재 코스닥 기관 억 순매도', '종합 경제 정보 미디어 데 일리 무단 전재 재 배포 금지', '전국 적 인 소비 붐 조성 기여 할 예정', '데일리 권오석 기자 중소 벤처 기업 부이 하 중기 부 대한민국 동행 세일 개 홈 쇼핑 사 홍보 판매 동참 한다고 일 밝혔 다', '대한민국 동행 세일 라이브 커머스 언 택트 콘서트 행사 연계 등 비 대면 라는 새로운 형태 소비 촉진 행사 다']
['실적 기반 저가 매집 해야 할 월 급등 유망주 전격 공개', '하이스 탁론 선취 수수료 없 월 최저 금리 상품 출시', '실적 기반 저가 매집 해야 할 월 급등 유망주 전격 공개', '하이스 탁론 선취 수수료 없 월 최저 금리 상품 출시', '실적 기반 저가 매집 해야 할 월 급등 유망주 전격 공개']


In [65]:
# tf = CountVectorizer()
tf = TfidfVectorizer()

In [66]:
X_train_fake = tf.fit_transform(fake)
X_train_true = tf.transform(true)
X_test = tf.transform(test['content'])
# X_test = tf.fit_transform(tst)

print(X_train_fake.shape)
print(X_train_true.shape)
print(X_test.shape)

(46932, 8880)
(71813, 8880)
(142565, 8880)


In [73]:
a = tf.get_feature_names()
print(a)

['andererapos', 'apos', 'click', 'cm', 'cs', 'dition', 'ealthcare', 'ed', 'est', 'forestmt', 'httpbattletomatocokr', 'httpbitlyru', 'httpbitlyvalueline', 'httpsgooglsk', 'httpvalueitoozacom', 'httpwwwbujanecokr', 'httpwwwitoozacom', 'ie', 'ihone', 'ime', 'img', 'inech', 'intro', 'itoozanews', 'kg', 'lick', 'mg', 'mm', 'nbsp', 'nbspnbsp', 'nbspnbspnbspnbsp', 'nt', 'oney', 'ooo', 'op', 'pipe', 'ppi', 'pt', 'quot', 'rone', 'sin', 'teel', 'ttpwwwasiaecokreventbrand', 'ttpwwwhislcokrmainmainphp', 'vip', 'vs', 'wwwhislcokriadmainmainphp', 'wwwinfostockcokr', 'yungcomelectioncandidates', '가게', '가격', '가결', '가계', '가계약', '가고시마', '가공', '가관', '가구', '가까운', '가까이', '가나메', '가난', '가능', '가다', '가동', '가득', '가량', '가려', '가려낸다', '가려진', '가로', '가로막', '가리', '가마솥', '가만', '가망', '가뭄', '가비아', '가사', '가상', '가상현실', '가세', '가속', '가속기', '가솔린', '가수', '가스', '가스관', '가스전', '가스층', '가슴', '가시', '가압류', '가요', '가운데', '가을', '가이드', '가입', '가입비', '가입자', '가장', '가전', '가전제품', '가점', '가정', '가정폭력', '가져', '가져가', '가져간', '가져갑니다', '가져도', '가족', 

In [57]:
tf.vocabulary_

{'데일리': 9377,
 '현재': 34170,
 '코스닥': 30797,
 '기관': 5894,
 '순매도': 18481,
 '실적': 19581,
 '기반': 5984,
 '저가': 26327,
 '매집': 11726,
 '해야': 33784,
 '급등': 5829,
 '유망주': 23555,
 '전격': 26492,
 '공개': 4368,
 '하이스': 33193,
 '탁론': 31187,
 '선취': 17278,
 '수수료': 18293,
 '최저': 29987,
 '금리': 5744,
 '상품': 16698,
 '출시': 30184,
 '종합': 27815,
 '경제': 3895,
 '정보': 27009,
 '미디어': 12840,
 '일리': 25232,
 '무단': 12439,
 '전재': 26695,
 '배포': 14052,
 '금지': 5792,
 '전국': 26508,
 '소비': 17817,
 '조성': 27547,
 '기여': 6068,
 '예정': 22057,
 '권오석': 5397,
 '기자': 6097,
 '중소': 28201,
 '벤처': 14418,
 '기업': 6059,
 '부이': 15103,
 '중기': 28167,
 '대한민국': 9204,
 '동행': 9876,
 '세일': 17609,
 '쇼핑': 18170,
 '홍보': 34443,
 '판매': 32084,
 '동참': 9856,
 '한다고': 33373,
 '밝혔': 13805,
 '라이브': 10627,
 '커머스': 30638,
 '택트': 31340,
 '콘서트': 30850,
 '행사': 33911,
 '연계': 21577,
 '대면': 9016,
 '라는': 10578,
 '새로운': 16738,
 '형태': 34267,
 '촉진': 29772,
 '권역': 5387,
 '에서': 21233,
 '현장': 34168,
 '오프라인': 22314,
 '판촉': 32118,
 '통해': 31624,
 '심리': 19633,
 '진작': 28827,
 '위함':

In [68]:
X_train_fake
X_train_true
feature_matrix = []
feature_matrix = X_train_fake.toarray()
#전체 문서에서 각 단어별 등장 횟수
word_count = feature_matrix.sum(axis=0)

# 단어의 이름 저장
word_name = tf.get_feature_names()
word_dic = {}

for i in range(len(word_name)):
  if word_count[i]>10:
    word_dic[word_name[i]] = word_count[i]



# def feature_ext(n):
#   feature_matrix = []
#   feature_matrix = X_train_fake
# def feature ext(n):
# vectorizer = CountVectorizer(min df,max features)
# X1 = vectorizer.ﬁt transform(전처리 된 데이터)
# a1 = [ ]
# a1 = X1.toarray()
# b1 = a1.sum(axis=0)
# c1 = vectorizer.get feature names()
# d1 = {}
# for i in range(len(c1)):
# if b1[i] > n:
# d1[c1[i]] = b1[i]
# return(d1)


In [71]:
print(word_dic)

{'apos': 12.035852858031776, 'est': 105.3603294587846, 'nbspnbspnbspnbsp': 205.11932324518222, '가능': 1877.4085403479937, '가장': 16.989751049361967, '갈까': 11.902739692198683, '갈아탈': 165.94131674557647, '감염': 14.768056864825173, '강남': 18.26944572254569, '강력': 13.20949728698019, '강세': 37.19550252813185, '개념': 24.212361706112063, '개미': 32.00442787417927, '개발': 36.50864562538165, '개별': 211.77875827865822, '개요': 110.9139121203568, '개월': 43.60509249170524, '개인': 39.42213593458907, '개장': 22.873012926193297, '갤럭시': 13.728629215004164, '거래': 30.252689872677674, '거래량': 708.4748322920485, '검사': 19.450877040712108, '게임': 11.916595837760418, '결과': 16.16751946463899, '결국': 38.162611892158964, '결단': 14.480829776105217, '결정': 22.42895906909311, '결코': 111.54445486253854, '경고': 34.049594549122354, '경기': 11.026673942341414, '경제': 32.16639942792042, '계약': 15.691161962131167, '계좌': 16.611621079346676, '고서': 14.806934394374581, '고수': 14.267294681921394, '골든': 10.372535812409467, '공개': 1584.129724920948, '공급':

핵심문장을 textRank로 도출한 후, 각 content를 해당 핵심문장과 비교하면?